In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver import Chrome
from time import sleep
from tqdm import tqdm
import pickle

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


In [3]:
base_url = "https://www.doximity.com/residency/specialties"

In [4]:
with open("parse.txt") as f:
  data = f.read()
f.close()

In [5]:
# sourceCode, driver = parse2(base_url)
soup = BeautifulSoup(data)
# mydivs = soup.find_all("li")#, {"class": "residency-specialties-directory-list-item"})
# print(mydivs)

In [6]:
specialties = set([])
mydivs = soup.find_all("li")#, {"class": "residency-specialties-directory-list-item"})
for li in mydivs:
    a= li.find_all("a")
    text = a[0].text.split("\n")[1].split(" ")
    text = " ".join([x for x in text if x!=''])
    # print(text)
    specialties.add((text,a[0]['href'] ))


In [7]:
specialty_to_location = {}
specialty_url = "https://www.doximity.com"
for specialty in tqdm(specialties):
    specialty_to_location[specialty[0]] = []
    data = requests.get(specialty_url+specialty[1])
    soup = BeautifulSoup(data.text, 'html.parser')
    mydivs = soup.find_all('div', {"class": "residency-institution-directory-content"})
    myli = mydivs[0].find_all("li")
    for x in myli:
        a = x.find_all("a")
        specialty_to_location[specialty[0]].append(a[0]['href'])


100%|██████████| 28/28 [00:29<00:00,  1.06s/it]


In [9]:
specialty_state_positions = {}
print(len(specialty_to_location.keys()))
counter = 0
for specialty in specialty_to_location.keys():
    print(counter, specialty)
    locations = specialty_to_location[specialty]
    specialty_state_positions[specialty] = {}
    for location in tqdm(locations):
        data = requests.get(specialty_url+location)
        soup = BeautifulSoup(data.text, 'html.parser')
        mydivs = soup.find_all('div', {"data-residency-positions-per-cycle=": ""})

        try:
            mydivs = mydivs[0].find_all('main',{"class", "page"})#,  class="residency-program-metric-icon"
            mydivs = mydivs[0].find_all('div', {"class": "residency-program-details-show"})
            mydivs = mydivs[0].find_all('div', {"class": "fs-unmask"})
            mydivs = mydivs[0].find_all('div', {"class": "residency-program-metric-container"})
            mydivs = mydivs[0].find_all('strong', {"class": "metric-box-number"})
            positions = mydivs[0].text.split('\n')[1].split("  ")[-1].split(' ')[0]

            mydivs = soup.find_all('div', {"data-residency-positions-per-cycle=": ""})
            mydivs = mydivs[0].find_all('main',{"class", "page"})#,  class="residency-program-metric-icon"
            mydivs = mydivs[0].find_all('div', {"class": "residency-program-details-show"})
            mydivs = mydivs[0].find_all('div', {"class": "fs-unmask"})
            mydivs = mydivs[0].find_all('p', {"class": "residency-program-hero-subtitle"})
            state = mydivs[0].text.split('\n')[1].split(' ')[-1]

            if state not in specialty_state_positions[specialty]:
                specialty_state_positions[specialty][state] = int(positions)
            else:
                specialty_state_positions[specialty][state] += int(positions)
        except:
            print(specialty, specialty_url+location)
    counter += 1
        

28
0 Physical Medicine & Rehabilitation


100%|██████████| 100/100 [00:54<00:00,  1.83it/s]


1 Pediatrics


100%|██████████| 212/212 [01:55<00:00,  1.84it/s]


2 Child Neurology


...

 63%|██████▎   | 124/197 [01:03<00:41,  1.75it/s]

Radiology (Diagnostic) https://www.doximity.com/residency/programs/7a0b2d78-6201-478c-be3a-252c705789d2-temple-university-hospital-radiology-diagnostic


100%|██████████| 197/197 [01:40<00:00,  1.97it/s]


27 Thoracic Surgery (Integrated)


100%|██████████| 34/34 [00:18<00:00,  1.86it/s]


In [10]:
with open('specialty_state_positions.pickle', 'wb') as handle:
    pickle.dump(specialty_state_positions, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
specialty_state_positions

{'Physical Medicine & Rehabilitation': {'NY': 86,
  'PA': 53,
  'TX': 38,
  'MI': 28,
  'NC': 14,
  'OH': 23,
  'VA': 13,
  'GA': 6,
  'NV': 5,
  'AZ': 2,
  'IN': 4,
  'NJ': 14,
  'MD': 17,
  'FL': 39,
  'CA': 32,
  'LA': 6,
  'IL': 25,
  'MN': 12,
  'DC': 6,
  'WI': 9,
  'WA': 11,
  'MA': 14,
  'AL': 4,
  'AR': 4,
  'CO': 6,
  'CT': 4,
  'KS': 4,
  'KY': 9,
  'MO': 8,
  'NE': 4,
  'NM': 5,
  'PR': 6,
  'UT': 6,
  'TN': 4},
 'Pediatrics': {'FL': 193,
  'IL': 154,
  'NY': 482,
  'PA': 152,
  'MI': 126,
  'IN': 35,
  'NJ': 92,
  'GA': 71,
  'TX': 247,
  'RI': 18,
  'VA': 86,
  'NC': 76,
  'OH': 190,
  'IA': 23,
  'WV': 19,
  'CA': 328,
  'MA': 89,
  'DC': 51,
  'MO': 85,
  'NH': 7,
  'TN': 68,
  'ID': 4,
  'PR': 32,
  'MD': 64,
  'NV': 13,
  'LA': 50,
  'WA': 51,
  'ME': 6,
  'WI': 47,
  'MN': 37,
  'SC': 41,
  'OK': 44,
  'OR': 18,
  'AZ': 62,
  'DE': 22,
  'HI': 17,
  'AL': 39,
  'AR': 25,
  'CO': 33,
  'CT': 41,
  'KS': 5,
  'KY': 41,
  'MS': 16,
  'NE': 15,
  'NM': 21,
  'SD': 6,
  '

In [98]:
mydivs = soup.find_all('div', {"data-residency-positions-per-cycle=": ""})
mydivs = mydivs[0].find_all('main',{"class", "page"})#,  class="residency-program-metric-icon"
mydivs = mydivs[0].find_all('div', {"class": "residency-program-details-show"})
mydivs = mydivs[0].find_all('div', {"class": "fs-unmask"})
mydivs = mydivs[0].find_all('div', {"class": "residency-program-metric-container"})
mydivs = mydivs[0].find_all('strong', {"class": "metric-box-number"})

mydivs[0].text.split('\n')[1].split("  ")[-1].split(' ')[0]

'2'

In [103]:
mydivs = soup.find_all('div', {"data-residency-positions-per-cycle=": ""})
mydivs = mydivs[0].find_all('main',{"class", "page"})#,  class="residency-program-metric-icon"
mydivs = mydivs[0].find_all('div', {"class": "residency-program-details-show"})
mydivs = mydivs[0].find_all('div', {"class": "fs-unmask"})
mydivs = mydivs[0].find_all('p', {"class": "residency-program-hero-subtitle"})


mydivs[0].text.split('\n')[1].split(' ')[-1]

'OH'